In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import matplotlib.pyplot as plt

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir /content/runs

In [ ]:
#Define transforms for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),  # Adding color jitter
    transforms.ToTensor(),
])

# Load FashionMNIST dataset
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

# Load pre-trained ResNet101 model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device:", device)
resnet = models.resnet101(pretrained=True)

# Freeze all layers in the pre-trained ResNet101 model
for param in resnet.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to match the number of classes in FashionMNIST (10)
num_ftrs = resnet.fc.in_features
resnet.fc = nn.Linear(num_ftrs, 10)
resnet.to(device)

In [ ]:
# Define loss function
criterion = nn.CrossEntropyLoss()

# Define the list of optimizers to use
optimizers = [optim.Adam(resnet.parameters()),
              optim.Adagrad(resnet.parameters()),
              optim.RMSprop(resnet.parameters())]

# Lists to store training statistics
train_losses = [[] for _ in range(len(optimizers))]
train_accuracy = [[] for _ in range(len(optimizers))]

In [ ]:
# Training loop
from tqdm import tqdm
for i, optimizer in enumerate(optimizers):
    print(f"Training with optimizer: {type(optimizer).__name__}")
    for epoch in range(1):  # Train for 1 epochs
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in tqdm(trainloader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = resnet(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_losses[i].append(running_loss / len(trainloader))
        train_accuracy[i].append(100 * correct / total)
        print(f"Epoch [{epoch + 1}/5], Loss: {train_losses[i][-1]:.4f}, Accuracy: {train_accuracy[i][-1]:.2f}%")

In [ ]:
# # Plot training loss and accuracy curves
# plt.figure(figsize=(12, 6))
# for i in range(len(optimizers)):
#     plt.subplot(2, len(optimizers)//2, i+1)
#     plt.plot(train_losses[i], label='Loss')
#     plt.xlabel('Epoch')
#     plt.ylabel('Training Loss')
#     plt.title(f'Training Loss with {type(optimizers[i]).__name__}')
#     plt.legend()

# plt.figure(figsize=(12, 6))
# for i in range(len(optimizers)):
#     plt.subplot(2, len(optimizers)//2, i+1)
#     plt.plot(train_accuracy[i], label='Accuracy')
#     plt.xlabel('Epoch')
#     plt.ylabel('Training Accuracy')
#     plt.title(f'Training Accuracy with {type(optimizers[i]).__name__}')
#     plt.legend()

# plt.show()

import matplotlib.pyplot as plt

# Assuming train_losses is a list of lists containing training losses for each optimizer
# train_losses = [adam_losses, adagrad_losses, adadelta_losses, rmsprop_losses]

plt.figure(figsize=(15, 6))
num_optimizers = len(train_losses)

for i in range(num_optimizers):
    plt.subplot(2, (num_optimizers + 1) // 2, i + 1)  # Adjusted to ensure integer number of subplots
    plt.plot(train_losses[i], label='Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Training Loss - {optimizers[i]}')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# # Testing
# correct = 0
# total = 0
# with torch.no_grad():
#     for images, labels in testloader:
#         images, labels = images.to(device), labels.to(device)
#         outputs = resnet(images)
#         _, predicted = outputs.max(1)
#         total += labels.size(0)
#         correct += predicted.eq(labels).sum().item()

# print(f"Top-5 Test Accuracy: {(100 * correct / total):.2f}%")

import torch

# Assuming testloader is the DataLoader for the FashionMNIST test set
correct = 0
total = 0
top5_correct = 0

# Set model to evaluation mode
resnet.eval()

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = resnet(images)
        _, predicted = torch.topk(outputs, 5, dim=1)  # Get top-5 predictions
        total += labels.size(0)
        correct += (predicted == labels.view(-1, 1)).sum().item()
        for i in range(labels.size(0)):
            if labels[i] in predicted[i]:
                top5_correct += 1

print(f'Top-5 Test Accuracy: {100 * top5_correct / total:.2f}%')
